In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import gc
import glob

In [2]:
def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)

In [3]:
Q1_start="2019-02-03"
Q1_end="2019-05-04"

daily_item_files_2019Q1=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_item_files_2019Q1=[x for x in daily_item_files_2019Q1 if x[-4:]==".txt" and "daily" in x.lower()]
daily_item_files_2019Q1=[x for x in daily_item_files_2019Q1 if x.split("s/MediaStorm_")[1][:10]>=Q1_start]
daily_item_files_2019Q1=[x for x in daily_item_files_2019Q1 if x.split("s/MediaStorm_")[1][:10]<=Q1_end]

print(len(daily_item_files_2019Q1))

daily_item_files_2019Q1.remove("/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-09/MediaStormDailySales20190212-122138-494.txt")
daily_item_files_2019Q1.append("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20190209.txt")
daily_item_files_2019Q1.sort()
daily_item_files_2019Q1
print(len(daily_item_files_2019Q1))


13
13


In [4]:
df_output_Q1_sales=pd.DataFrame()
for file in daily_item_files_2019Q1:
    try:
        week_end_dt=file.split("s/MediaStorm_")[1][:10]
    except:
        week_end_dt=file.split("rmDailySalesHistory")[1][:8]
        week_end_dt=week_end_dt[:4]+"-"+week_end_dt[4:6]+"-"+week_end_dt[6:]
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt', 'transaction_id',
                                                              'customer_id_hashed', 'item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    # Rewards
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_rewards_sales=df_rewards.groupby(["location_id"])['item_transaction_amt'].sum().to_frame().reset_index()
    df_rewards_sales=df_rewards_sales.rename(columns={"item_transaction_amt":"sales"})
    df_rewards_trans=df_rewards[['location_id','transaction_dt', 'transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans['trans']=1
    df_rewards_trans=df_rewards_trans.groupby(["location_id"])['trans'].sum().to_frame().reset_index()
    
    df_rewards=pd.merge(df_rewards_trans,df_rewards_sales,on="location_id",how="outer")
    df_rewards['rewards_label']="rewards"
    
    # Non Rewards
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    df_non_rewards_sales=df_non_rewards.groupby(["location_id"])['item_transaction_amt'].sum().to_frame().reset_index()
    df_non_rewards_sales=df_non_rewards_sales.rename(columns={"item_transaction_amt":"sales"})
    df_non_rewards_trans=df_non_rewards[['location_id','transaction_dt', 'transaction_id']].drop_duplicates()
    df_non_rewards_trans['trans']=1
    df_non_rewards_trans=df_non_rewards_trans.groupby(["location_id"])['trans'].sum().to_frame().reset_index()  
    
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards_trans,on="location_id",how="outer")
    df_non_rewards['rewards_label']="non_rewards"
    
    df=df_rewards.append(df_non_rewards)
    df['store_type']=np.where(df['location_id']=="6990","online","instore")
    df=df.groupby(["rewards_label","store_type"])['sales','trans'].sum().reset_index()
    df['week_end_dt']=week_end_dt
    
    df_output_Q1_sales=df_output_Q1_sales.append(df)
    print(datetime.datetime.now(),file)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


2019-10-29 13:31:45.802330 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-16/MediaStormDailySales20190219-113605-481.txt
2019-10-29 13:32:42.634493 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-02-23/MediaStormDailySales20190226-112611-940.txt
2019-10-29 13:36:09.462571 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-02/MediaStormDailySales20190305-112738-899.txt
2019-10-29 13:37:08.317518 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-09/MediaStormDailySales20190312-121205-996.txt
2019-10-29 13:39:05.549719 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-16/MediaStormDailySales20190319-112334-057.txt
2019-10-29 13:40:11.866338 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-23/MediaStormDailySales20190326-112740-903.txt
2019-10-29 13:41:36.855135 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-03-30/MediaStormDailySales20190402-112556-551.txt
2019-10-29 13:42:48.004720 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-04-06/MediaStormDailySales20190409-122759-

In [30]:
writer=pd.ExcelWriter("./BL_Q1_total_sales_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_Q1_sales.to_excel(writer,"Q1_total_sales",index=False)
writer.save()

In [24]:
df_non_rewards_sales.head(2)

,location_id,sales
0,1,29412.06
1,1001,13618.42


In [25]:
df_non_rewards_sales[df_non_rewards_sales["location_id"]!="6990"]['sales'].sum()

30958359.470000032

In [26]:
df_rewards_sales[df_rewards_sales["location_id"]!="6990"]['sales'].sum()

49260397.840000086

In [14]:
df_output_Q1_sales[df_output_Q1_sales['store_type']=="instore"]['trans'].sum()

36367502

In [13]:
(1277786880-1278058687.8500004)/1278058687.8500004


-0.000212672432482445

In [15]:
(35784041-36367502)/36367502

-0.016043471998709178

In [18]:
31183434.96000003/84313943

0.36984908842420083

In [19]:
df_rewards_sales['sales'].sum()

49587586.72000007

In [27]:
30958359.470000032+49260397.840000086

80218757.31000012

In [29]:
80218757.31000012/80212615

1.00007657536162